In [1]:
#!pip install folium geopy
#!pip install geojson
import pandas as pd
import re
from math import sin, asin, cos, radians, fabs, sqrt
from geojson import Point, Feature, FeatureCollection, dump

In [2]:
sensor_all = pd.read_csv('Pedestrian_Counting_System_-_Sensor_Locations.csv')
sensor_all.head()

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,59,Building 80 RMIT,RMIT_T,2019/02/13,A,NaN,North,South,-37.808256,144.963049,"(-37.80825648, 144.96304859)"
1,23,Spencer St-Collins St (South),Col623_T,2013/09/02,A,NaN,East,West,-37.819093,144.954527,"(-37.81909256, 144.95452749)"
2,20,Chinatown-Lt Bourke St (South),LtB170_T,2013/09/06,A,NaN,East,West,-37.811729,144.968247,"(-37.81172913, 144.9682466)"
3,34,Flinders St-Spark La,Fli32_T,2014/06/08,A,NaN,East,West,-37.815380,144.974150,"(-37.81537985, 144.9741505)"
4,57,Bourke St Bridge,BouBri_T,2018/08/13,A,NaN,West,East,-37.817673,144.950256,"(-37.8176735, 144.95025595)"


In [3]:
mel_all = pd.read_csv('mel.csv')
mel_all.head()

,the_geom,name,features,council_re,location_d
0,MULTIPOLYGON (((144.984881858222 -37.838765747...,Fawkner Park North East,"Chin up bar, parallel bars, balance beam, some...",1450806,Near Toorak Road
1,MULTIPOLYGON (((144.973011582673 -37.820142955...,Riverslide Skate Park,"Concrete skate ramp, public toilet.",1450832,NaN
2,MULTIPOLYGON (((144.930838670413 -37.795482932...,Bellair Street Reserve Playground,"Spring rocker, swings, platform, slide, picnic...",1450809,NaN
3,MULTIPOLYGON (((144.962101292482 -37.781312338...,Princes Park North Playground,"Swing, spring riders, see saw.",1450816,NaN
4,MULTIPOLYGON (((144.940758503705 -37.820191736...,Buluk Park Playground,NaN,1588995,NaN


In [4]:
mel_all = mel_all.drop(columns = ['council_re','location_d'])
mel_all.head()

,the_geom,name,features
0,MULTIPOLYGON (((144.984881858222 -37.838765747...,Fawkner Park North East,"Chin up bar, parallel bars, balance beam, some..."
1,MULTIPOLYGON (((144.973011582673 -37.820142955...,Riverslide Skate Park,"Concrete skate ramp, public toilet."
2,MULTIPOLYGON (((144.930838670413 -37.795482932...,Bellair Street Reserve Playground,"Spring rocker, swings, platform, slide, picnic..."
3,MULTIPOLYGON (((144.962101292482 -37.781312338...,Princes Park North Playground,"Swing, spring riders, see saw."
4,MULTIPOLYGON (((144.940758503705 -37.820191736...,Buluk Park Playground,NaN


In [23]:
equipment = list(mel_all['features'])
unique_equipment = {}
for i in equipment :
    index = equipment.index(i)
    if type(i) == str :
        temp = i.replace('.','').lower().split(', ')
        for j in temp :
            if j not in unique_equipment :
                unique_equipment[j] = []
                unique_equipment[j].append(index)
            else :
                unique_equipment[j].append(index)

In [24]:
for k,v in unique_equipment.items() :
    temp = [0] * len(equipment)
    for each in v :
        temp[each] = 1
    mel_all[k] = temp

In [25]:
mel_all.head()

,the_geom,name,features,chin up bar,parallel bars,balance beam,some shade,concrete skate ramp,public toilet,spring rocker,...,wooden tower,stepping posts,grassed areas,rock wall,monkey bars,tunnels and barrels temporary basketball court with backboard and ring,basketball rings,synthetic surface,dragon slide,giraffe swing
0,MULTIPOLYGON (((144.984881858222 -37.838765747...,Fawkner Park North East,"Chin up bar, parallel bars, balance beam, some...",1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MULTIPOLYGON (((144.973011582673 -37.820142955...,Riverslide Skate Park,"Concrete skate ramp, public toilet.",0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,MULTIPOLYGON (((144.930838670413 -37.795482932...,Bellair Street Reserve Playground,"Spring rocker, swings, platform, slide, picnic...",0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,MULTIPOLYGON (((144.962101292482 -37.781312338...,Princes Park North Playground,"Swing, spring riders, see saw.",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MULTIPOLYGON (((144.940758503705 -37.820191736...,Buluk Park Playground,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
geom_lat = []
geom_lon = []
geom_name = []
geom_point = []
lon_info = r'\d\d\d\.\d+'
lat_info = r'\-\d+\.\d+'
point_info = r'(\d+.\d+) (\-\d+\.\d+)'
for i in range(len(equipment)) : 
    lat_list = re.findall(lat_info,list(mel_all['the_geom'])[i])
    geom_point += re.findall(point_info,list(mel_all['the_geom'])[i])
    geom_lon += re.findall(lon_info,list(mel_all['the_geom'])[i])
    geom_name += [list(mel_all['name'])[i]] * len(lat_list)
    geom_lat += lat_list
geom_lat = list(map(float,geom_lat))
geom_lon = list(map(float,geom_lon))

In [9]:
sensor_id = list(sensor_all['sensor_id'])
sensor_lat = list(sensor_all['latitude'])
sensor_lon = list(sensor_all['longitude'])

In [10]:
def distance(lon1, lat1, lon2, lat2):
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    r = 6378 * 1000
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    result = r*c
    return result
distance_list = []
lowest_id = []
for index in range(len(geom_name)):
    temp = None
    temp_id = None
    for si in range(len(sensor_id)):
        d = distance(geom_lon[index],geom_lat[index],sensor_lon[si],sensor_lat[si])
        if temp == None or temp > d:
            temp = d
            temp_id = sensor_id[si]
    distance_list.append(temp)
    lowest_id.append(temp_id)
            

In [11]:
distance_df = pd.DataFrame({'lat':geom_lat,'lon':geom_lon,'name':geom_name,'distance':distance_list,'id':lowest_id})

In [12]:
sensor_unique_id = list(distance_df[distance_df.distance < 500].id.value_counts().keys())

In [18]:
list(distance_df[distance_df.distance < 200].id.value_counts().keys())

[8, 7, 46]

In [13]:
playground_name = []
sensor_playground_id = []
for i in sensor_point_id :
    p = list(distance_df[distance_df.id == i].name.value_counts().keys())
    sensor_playground_id += [i]*len(p)
    playground_name += p    

NameError: name 'sensor_point_id' is not defined

In [14]:
new_mel = mel_all[mel_all.name.isin(playground_name)]

In [71]:
new_mel

,the_geom,name,features,chin up bar,parallel bars,balance beam,some shade,concrete skate ramp,public toilet,spring rocker,...,wooden tower,stepping posts,grassed areas,rock wall,monkey bars,tunnels and barrels temporary basketball court with backboard and ring,basketball rings,synthetic surface,dragon slide,giraffe swing
1,MULTIPOLYGON (((144.973011582673 -37.820142955...,Riverslide Skate Park,"Concrete skate ramp, public toilet.",0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,MULTIPOLYGON (((144.947160992534 -37.821930981...,Docklands Park Playground,"Carousels, platforms, swing, slide, public toi...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,MULTIPOLYGON (((144.944909035056 -37.820112294...,Victoria Green Playground,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,MULTIPOLYGON (((144.962585326539 -37.802855460...,Lincoln Square Playground,"Swings, spinners, double slide, public toilet,...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,MULTIPOLYGON (((144.95476596702 -37.8094592916...,Flagstaff Gardens Playground,"Sandpit with diggers, spiral slide, swings, sa...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,MULTIPOLYGON (((144.971633515875 -37.818069284...,Birrarung Marr Artplay,"Rock and rope climbing features, swinging hamm...",0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
16,MULTIPOLYGON (((144.941286023475 -37.823008440...,Point Park Playground,"Bells, Musical Wall, Sand pit, Swing, Carousel...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,MULTIPOLYGON (((144.969754004322 -37.795185277...,Neill Street Reserve,1 x 5 way swing,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,MULTIPOLYGON (((144.970855853144 -37.802002782...,Carlton Gardens Playground,"Cubby with slide, swings, track glide, sandpit...",0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
31,MULTIPOLYGON (((144.97407820177 -37.7969954259...,Station Street Park Playground,Climbers x 3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
new_mel.loc[new_mel.name=='Victoria Green Playground','slide'] = 1
new_mel.loc[new_mel.name=='Victoria Green Playground','sandpit'] = 1
new_mel.loc[new_mel.name=='Victoria Green Playground','climbing frame'] = 1

/Users/xuweizhe/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [102]:
new_mel.loc[new_mel.name=='Victoria Green Playground','climbing frame']

7    1
Name: climbing frame, dtype: int64

In [98]:
'climbs' in list(unique_equipment.keys())

False

In [99]:
list(unique_equipment.keys())

['chin up bar',
 'parallel bars',
 'balance beam',
 'some shade',
 'concrete skate ramp',
 'public toilet',
 'spring rocker',
 'swings',
 'platform',
 'slide',
 'picnic table',
 'swing',
 'spring riders',
 'see saw',
 'small tower',
 'shade structure',
 'stockyard style fencing',
 'carousels',
 'platforms',
 'tower platforms',
 'sandpits',
 'rockers',
 '5-way swing',
 'boardwalk',
 'fitness track',
 'barbecue',
 'spinners',
 'double slide',
 'lots of shade',
 'sandpit with diggers',
 'spiral slide',
 'sandpit',
 'balancing bridges',
 'shade',
 'rock and rope climbing features',
 'swinging hammocks',
 'liberty swing for wheelchairs',
 'quiet areas and active play areas',
 'disabled access',
 'sheep sculptures',
 'combination units',
 'barbeque',
 'multi-level play structure',
 'climbers',
 'see-saw',
 'cubby',
 'picnic tables',
 'bells',
 'musical wall',
 'sand pit',
 'carousel and slide',
 'skate park',
 'bmx track',
 'toddler playspace - slide',
 '1 x 5 way swing',
 'spring rider',
 '

In [104]:
distance_df.to_csv('playground_location.csv')